# Binary Sentiment Analysis of French Movie Reviews

### Objectives
1. Text cleaning
2. Text preprocessing for custom embedding Neural Network
3. Train RNN model for sentiment analysis

⚠️ This notebook will be your final deliverable. 
- Make sure it can run "restart and run all"
- Delete useless code cells
- Do not "clear output"

# 0. Load data

Our dataset contains 30,000 french reviews of movies, along with the binary class 1 (positive) or 0 (negative) score

In [1]:
import pandas as pd

In [2]:
# We load the dataset for you
data = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies.csv')
data

review  polarity
0      Ça commence à devenir énervant d'avoir l'impre...         0
1      J'ai aimé ce film, si il ressemble a un docume...         1
2      Une grosse merde ce haneke ce faire produire p...         0
3      Beau mélodrame magnifiquement photographié, "V...         1
4      A la poursuite du diamant vers est un film pro...         1
...                                                  ...       ...
29946  Le meilleur film de super-héros derrière le ba...         1
29947  Un drame qui est d'une efficacité remarquable....         1
29948  Une daube hollywoodienne de plus, aucun intérê...         0
29949  Et voilà un nouveau biopic sur la star du X Li...         0
29950  Un film qui fait vieux, avec des acteurs pas t...         0

[29951 rows x 2 columns]

In [3]:
# We create features
y = data.polarity
X = data.review

# We analyse class balance
print(pd.value_counts(y))

1    15051
0    14900
Name: polarity, dtype: int64


In [4]:
# We check various reviews
print(f'polarity: {y[0]} \n')
print(X[0])

polarity: 0 

Ça commence à devenir énervant d'avoir l'impression de voir et revoir le même genre de film à savoir : la comédie romantique, surement le genre le plus prolifique de le production française actuelle. Le problème c'est que l'on a souvent affaire à des niaiseries de faible niveau comme celui ci. Avec un scenario ultra balisé et conventionnel, c'est à se demander comment ça peut passer les portes d'un producteur. Bref cette sempiternel histoire d'un homme mentant au nom de l'amour pour reconquérir une femme et qui à la fin se prend son mensonge en pleine figure est d'une originalité affligeante, et ce n'est pas la présence au casting de l'ex miss météo Charlotte Le Bon qui rêve surement d'avoir la même carrière que Louise Bourgoin qui change la donne.


# 1. Clean Text

❓ We need to give a _quick & dirty_ cleaning to all the sentences in the dataset. Create a variable `X_clean` of similar shape, but with the following cleaning:
- Replace french accents by their non-accentuated equivalent using the [unidecode.unidecode()](https://pypi.org/project/Unidecode/) method
- Reduce all uppercases to lowercases
- Remove any characters outside of a-z, for instance using `string.isalpha()`

😌 You will be given the solution `X_clean` in the next question to make sure you can complete the challenge

In [5]:
### YOUR CODE
import unidecode
def encode(text):
    no_accent = unidecode.unidecode(text)
    return no_accent
data['clean_text']=data.review.apply(lambda x:encode(x))

In [6]:
def lowercase(text): 
    lowercased = text.lower() 
    return lowercased

data['clean_text'] = data.clean_text.apply(lowercase)


In [7]:
import re
def remove_nonalpha(text):
    text = ''.join(c for c in text if c.isalpha() or c == ' ')
    return re.sub(" +", " ", text)
X_clean = data.clean_text.apply(lambda text: remove_nonalpha(text))

In [8]:
from nbresult import ChallengeResult

result = ChallengeResult('C14',
    shape = X_clean.shape,
    first_sentence = X_clean[0]
)
result.write()

# 2. Preprocess data

Now that we have clean sentences, we need to convert each one into a list of integers of fixed size
- For example, the sentence: `"this was good"` should become something like `array([1, 3, 18, 0, 0, 0, ...0], dtype=int32)` where each integer match to a each _unique_ word in your corpus of sentences.

❓ Create a numpy ndarray `X_input` of shape (29951, 100) that will be the direct input to your Neutral Network. 

- 29951 represents the number of reviews in the dataset `X_clean`
- 100 represents the maximum number of words to keep for each movie review.
- It must contain only numerical values, without any `NaN`
- In the process, compute and save the number of _unique_ words in your cleaned corpus under `vocab_size` variable

👉 First, you **must** start back from the clean solution below (14Mo)

In [9]:
X_clean = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_clean.csv")['review']
X_clean

0        ca commence a devenir enervant de voir et revo...
1        aime ce film si il ressemble a un documentaire...
2        une grosse merde ce haneke ce faire produire p...
3        beau melodrame magnifiquement photographie ver...
4        a la poursuite du diamant vers est un film pro...
                               ...                        
29946    le meilleur film de derriere le batman de nola...
29947    un drame qui est efficacite remarquable un fil...
29948    une daube hollywoodienne de plus aucun interet...
29949    et voila un nouveau biopic sur la star du x li...
29950    un film qui fait vieux avec des acteurs pas to...
Name: review, Length: 29951, dtype: object

In [10]:
### YOUR CODE
def convert_sentences(X):
    return [sentence.split(' ') for sentence in X]

X_clean = convert_sentences(X_clean)

In [11]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_clean, size=100)

In [12]:
import numpy as np

def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return embedded_sentence

In [13]:
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed


X_clean = embedding(word2vec, X_clean)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
bag_of_words = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False).fit_transform(splited_labels_from_corpus)

vec = CountVectorizer(max_features=100)
X_input = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False).fit_transform(X_clean)
X_input = X_input.toarray()

AttributeError: 'list' object has no attribute 'lower'

In [15]:
np.isnan(X_input).sum()

NameError: name 'X_input' is not defined

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('C1415',
    type_X = type(X_input),
    shape = X_input.shape, 
    input_1 = X_input[1], 
)
result.write()

# 3. Neural Network

❓Create and fit a Neural Netork that takes `X_input` and `y` as input, to binary classify each sentence's sentiment

- You cannot use transfer learning or other pre-existing Word2Vec models
- You must use a "recurrent" architecture to _capture_ a notion of order in the sentences' words
- The performance metrics for this task is "accuracy"
- Store your model in a variable `model` 
- Store the result your `model.fit()` in a variable `history`. 
- ⚠️ `history.history` must comprises a measure of the `val_accuracy` at each epoch.
- You don't need to cross-validate your model

😌 Don't worry, you will not be judged on your computer power: You should be able to reach accuracy significantly better than baseline in less than 3 minutes even without GPUs.

👉 But first, you **must** start back from the solution below (70Mo)

In [16]:
url = 'https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_input.csv'
X_input = np.genfromtxt(url, delimiter=',', dtype='int32')

In [38]:
y=data.polarity

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_input, y, test_size=0.3, random_state=42)

In [52]:
word_to_id = {}
iter_ = 1
for sentence in X_train:
    for word in sentence:
        if word in word_to_id:
            continue
        word_to_id[word] = iter_
        iter_ += 1

In [53]:
vocab_size = len(word_to_id)

In [59]:
def tokenize(sentences, word_to_id):
    return [[word_to_id[_] for _ in s if _ in word_to_id] for s in sentences]

X_token_train = tokenize(X_train, word_to_id)
X_token_test = tokenize(X_test, word_to_id)

In [60]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_token_train, dtype='float32', padding='post', value=0, maxlen=100)
X_test_pad = pad_sequences(X_token_test, dtype='float32', padding='post', value=0, maxlen=100)

In [76]:
from tensorflow.keras.layers import SpatialDropout1D

es = EarlyStopping(patience=10, restore_best_weights=True)
model = Sequential()

model.add(layers.Embedding(input_dim=X_train.max(), output_dim=100, mask_zero=True))
model.add(SpatialDropout1D(0.2))
model.add(layers.Conv1D(filters=64, kernel_size=4, activation='relu', padding='same'))
model.add(SpatialDropout1D(0.2))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])

history=model.fit(X_train_pad, y_train,epochs=30,  validation_split=0.3,batch_size=16, callbacks=[es])

Epoch 1/30
918/918 [==============================] - 28s 30ms/step - loss: 0.6796 - accuracy: 0.5407 - val_loss: 0.6606 - val_accuracy: 0.5697
Epoch 2/30
918/918 [==============================] - 28s 31ms/step - loss: 0.6552 - accuracy: 0.5716 - val_loss: 0.6598 - val_accuracy: 0.5707
Epoch 3/30
918/918 [==============================] - 28s 30ms/step - loss: 0.6499 - accuracy: 0.5868 - val_loss: 0.6588 - val_accuracy: 0.5591
Epoch 4/30
918/918 [==============================] - 27s 30ms/step - loss: 0.6465 - accuracy: 0.5780 - val_loss: 0.6579 - val_accuracy: 0.5710
Epoch 5/30
918/918 [==============================] - 27s 30ms/step - loss: 0.6418 - accuracy: 0.5890 - val_loss: 0.6599 - val_accuracy: 0.5594
Epoch 6/30
918/918 [==============================] - 27s 30ms/step - loss: 0.6400 - accuracy: 0.5878 - val_loss: 0.6592 - val_accuracy: 0.5599
Epoch 7/30
918/918 [==============================] - 27s 30ms/step - loss: 0.6357 - accuracy: 0.5990 - val_loss: 0.6593 - val_accuracy:

In [77]:
history.history

{'loss': [0.6711757779121399,
  0.6560041308403015,
  0.6512266993522644,
  0.6473015546798706,
  0.642941951751709,
  0.6402326226234436,
  0.6371654272079468,
  0.6345489025115967,
  0.6318196058273315,
  0.6299489140510559,
  0.6276379227638245,
  0.6254269480705261,
  0.623933732509613,
  0.6218879818916321],
 'accuracy': [0.5526991486549377,
  0.5724146962165833,
  0.5801717042922974,
  0.5791414380073547,
  0.587884247303009,
  0.5876426696777344,
  0.5940512418746948,
  0.5924978256225586,
  0.5968755483627319,
  0.59767746925354,
  0.598335325717926,
  0.6029000282287598,
  0.5975493788719177,
  0.6000191569328308],
 'val_loss': [0.6605668067932129,
  0.6598120331764221,
  0.658847987651825,
  0.6579153537750244,
  0.6598529815673828,
  0.6591825485229492,
  0.6592890620231628,
  0.6628679633140564,
  0.6594566702842712,
  0.6601935625076294,
  0.6623965501785278,
  0.6665889024734497,
  0.6691198945045471,
  0.6644095778465271],
 'val_accuracy': [0.5697243213653564,
  0.570732

In [78]:
from nbresult import ChallengeResult
result = ChallengeResult('C1517',
                         history=history.history)
result.write()